In [7]:
pip install pyodbc sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pyodbc
import re
from datetime import datetime

def execute_sql_batches(sql_file_path, batch_size=100):
    start_time = datetime.now()
    print(f"Execution started at: {start_time}")
    
    try:
        # 连接字符串
        conn = pyodbc.connect(
            'DRIVER={ODBC Driver 17 for SQL Server};'
            'SERVER=DESKTOP-VHHBRU7\\SQLEXPRESS;'
            'DATABASE=master;'
            'Trusted_Connection=yes;'
        )
        cursor = conn.cursor()

        # 读取 SQL 文件
        with open(sql_file_path, 'r', encoding='utf-8-sig') as f:
            sql_script = f.read()
        
        # 分割批次
        batches = [batch.strip() for batch in re.split(r'\bGO\b', sql_script, flags=re.IGNORECASE) if batch.strip()]
        total_batches = len(batches)
        
        print(f"Total batches to execute: {total_batches}")
        
        successful_batches = 0
        failed_batches = 0
        
        for i, batch in enumerate(batches, 1):
            try:
                if i % batch_size == 0 or i == total_batches:
                    print(f"Processing batch {i}/{total_batches}...")
                
                cursor.execute(batch)
                conn.commit()
                successful_batches += 1
                
            except pyodbc.Error as e:
                failed_batches += 1
                print(f"\nError in batch {i}:")
                print(f"SQL Error: {e}")
                print(f"Problematic SQL (first 200 chars):\n{batch[:200]}...")
                conn.rollback()
                
                # Optionally continue with next batch instead of breaking
                # continue
                # Or break if you want to stop on first error
                break

        end_time = datetime.now()
        duration = end_time - start_time
        
        print("\nExecution Summary:")
        print(f"Total batches: {total_batches}")
        print(f"Successful batches: {successful_batches}")
        print(f"Failed batches: {failed_batches}")
        print(f"Time elapsed: {duration}")
        
        return successful_batches == total_batches

    except Exception as e:
        print(f"Fatal error: {e}")
        return False
    finally:
        if 'conn' in locals():
            conn.close()

# 使用示例
if __name__ == "__main__":
    result = execute_sql_batches('instnwnd.sql', batch_size=50)
    print("Script completed successfully!" if result else "Script completed with errors")

Execution started at: 2025-09-05 23:59:58.672162
Total batches to execute: 402
Processing batch 50/402...
Processing batch 100/402...
Processing batch 150/402...
Processing batch 200/402...
Processing batch 250/402...
Processing batch 300/402...
Processing batch 350/402...
Processing batch 400/402...
Processing batch 402/402...

Execution Summary:
Total batches: 402
Successful batches: 402
Failed batches: 0
Time elapsed: 0:00:00.697656
Script completed successfully!


In [16]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install pandas

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ------------- -------------------------- 3.7/11.0 MB 22.4 MB/s eta 0:00:01
   ------------------------------------- -- 10.2/11.0 MB 28.3 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 26.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ----------------------- ---------------- 7.6/12.8 MB 35.1 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 33.1 MB/s eta 0:00:00

   ---------------------------------------- 0/4 [pytz]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   -------------------- ------------------- 2/4 [numpy]
   -------------------- ------------------- 2/4 [numpy]
   -------------------- ------------------- 2/4 [numpy]
   -------------------- ------------------- 2/4 [numpy]
   -------------

In [ ]:
import pyodbc
import pandas as pd

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-VHHBRU7\\SQLEXPRESS;'
    'DATABASE=Northwind;'
    'Trusted_Connection=yes;'
)

# read to DataFrame
df = pd.read_sql("""
    SELECT TOP 3 * FROM dbo.Employees
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print(df)

conn.close()



   EmployeeID   LastName FirstName                  Title TitleOfCourtesy  BirthDate   HireDate                        Address      City Region PostalCode Country       HomePhone Extension                                              Photo                                              Notes  ReportsTo                               PhotoPath
0           1    Davolio     Nancy   Sales Representative             Ms. 1948-12-08 1992-05-01  507 - 20th Ave. E.\r\nApt. 2A   Seattle     WA      98122     USA  (206) 555-9857      5467  b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...  Education includes a BA in psychology from Col...        2.0    http://accweb/emmployees/davolio.bmp
1           2     Fuller    Andrew  Vice President, Sales             Dr. 1952-02-19 1992-08-14             908 W. Capital Way    Tacoma     WA      98401     USA  (206) 555-9482      3457  b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...  Andrew received his BTS commercial in 1974 and...        NaN     http://acc

C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\2931652172.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""


In [5]:
import pyodbc
import pandas as pd
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-VHHBRU7\\SQLEXPRESS;'
    'DATABASE=Northwind;'
    'Trusted_Connection=yes;'
)

# read to DataFrame
df = pd.read_sql("""SELECT EmployeeID, LastName, FirstName FROM dbo.Employees
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print(df)

conn.close()

   EmployeeID   LastName FirstName
0           1    Davolio     Nancy
1           2     Fuller    Andrew
2           3  Leverling     Janet
3           4    Peacock  Margaret
4           5   Buchanan    Steven
5           6     Suyama   Michael
6           7       King    Robert
7           8   Callahan     Laura
8           9  Dodsworth      Anne


C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\3253942945.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""SELECT EmployeeID, LastName, FirstName FROM dbo.Employees


In [11]:
import pyodbc
import pandas as pd
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-VHHBRU7\\SQLEXPRESS;'
    'DATABASE=Northwind;'
    'Trusted_Connection=yes;'
)
df = pd.read_sql("""
                 SELECT * FROM dbo.Employees
                 """, conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print(df)

df2 = pd.read_sql("""
                 SELECT * FROM Employees
                 """, conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print(df2)

conn.close()

   EmployeeID   LastName FirstName                     Title TitleOfCourtesy  BirthDate   HireDate                           Address      City Region PostalCode Country       HomePhone Extension                                              Photo                                              Notes  ReportsTo                               PhotoPath
0           1    Davolio     Nancy      Sales Representative             Ms. 1948-12-08 1992-05-01     507 - 20th Ave. E.\r\nApt. 2A   Seattle     WA      98122     USA  (206) 555-9857      5467  b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...  Education includes a BA in psychology from Col...        2.0    http://accweb/emmployees/davolio.bmp
1           2     Fuller    Andrew     Vice President, Sales             Dr. 1952-02-19 1992-08-14                908 W. Capital Way    Tacoma     WA      98401     USA  (206) 555-9482      3457  b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...  Andrew received his BTS commercial in 1974 and...        

C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\2084663549.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""
C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\2084663549.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql("""


In [ ]:
import pyodbc
import pandas as pd
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-VHHBRU7\\SQLEXPRESS;'
    'DATABASE=Northwind;'
    'Trusted_Connection=yes;'
)

# read to DataFrame
df = pd.read_sql("""
                 SELECT distinct City  FROM dbo.Employees
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print('distinct City')
print(df)
print('\n')
print ('-------------------')
print('distinct Country')
# read to DataFrame
df = pd.read_sql("""
                 SELECT distinct Country  FROM dbo.Employees
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print(df)

print('\n')
print ('-------------------')
print('distinct postalcode, country')
# read to DataFrame
df = pd.read_sql("""
                 SELECT distinct postalcode,country  FROM dbo.Employees
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print(df)


conn.close()

distinct City
       City
0  Kirkland
1    London
2   Redmond
3   Seattle
4    Tacoma
-------------------
distinct Country
  Country
0      UK
1     USA


-------------------
distinct postalcode, country
  postalcode country
0      98033     USA
1      98052     USA
2      98105     USA
3      98122     USA
4      98401     USA
5    EC2 7JR      UK
6    RG1 9SP      UK
7    SW1 8JR      UK
8    WG2 7LT      UK


C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\838128117.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""
C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\838128117.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""
C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\838128117.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""


In [ ]:
import pyodbc
import pandas as pd
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-VHHBRU7\\SQLEXPRESS;'
    'DATABASE=Northwind;'
    'Trusted_Connection=yes;'
)

# read to DataFrame
df = pd.read_sql("""
                 SELECT distinct postalcode + ' '+ Country  FROM dbo.Employees
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print('distinct postalcode, country')
print(df)
print('\n')
print ('-------------------')
print('distinct postalcode and country')
# read to DataFrame
df = pd.read_sql("""
                 SELECT distinct postalcode + ' ' + Country as 'Postal Code and Country' From dbo.Employees
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print(df)
conn.close()

distinct postalcode, country
             
0   98033 USA
1   98052 USA
2   98105 USA
3   98122 USA
4   98401 USA
5  EC2 7JR UK
6  RG1 9SP UK
7  SW1 8JR UK
8  WG2 7LT UK
-------------------
distinct postalcode and country
  Postal Code and Country
0               98033 USA
1               98052 USA
2               98105 USA
3               98122 USA
4               98401 USA
5              EC2 7JR UK
6              RG1 9SP UK
7              SW1 8JR UK
8              WG2 7LT UK


C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\3782345982.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""
C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\3782345982.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""


In [27]:
import pyodbc

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-VHHBRU7\\SQLEXPRESS;'
    'DATABASE=Northwind;'
    'Trusted_Connection=yes;'
)

df = pd.read_sql("""
                    DECLARE @today DATETIME
                    SELECT @today = GETDATE()
                    SELECT @today
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print(df)
print ('-------------------')
# second variable

df = pd.read_sql("""
                    DECLARE @tod#ay2 DATETIME
                    SELECT @tod#ay2 = GETDATE()
                    SELECT @tod#ay2
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print(df)

conn.close()

                         
0 2025-09-06 00:54:59.347
-------------------
                         
0 2025-09-06 00:54:59.350


C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\1063101689.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""
C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\1063101689.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""


In [32]:
import pyodbc
import pandas as pd
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-VHHBRU7\\SQLEXPRESS;'
    'DATABASE=Northwind;'
    'Trusted_Connection=yes;'
)

# read to DataFrame
df = pd.read_sql("""
                 SELECT FirstName as [First \t Name] FROM dbo.Employees
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

print(df)
print('\n')
print ('-------------------')
print('')
df = pd.read_sql("""
                 SELECT FirstName as "First \n Name" FROM dbo.Employees
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

print(df)

conn.close()

  First \t Name
0         Nancy
1        Andrew
2         Janet
3      Margaret
4        Steven
5       Michael
6        Robert
7         Laura
8          Anne


-------------------

  First \n Name
0         Nancy
1        Andrew
2         Janet
3      Margaret
4        Steven
5       Michael
6        Robert
7         Laura
8          Anne


C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\1022755623.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""
C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\1022755623.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""


In [36]:
import pyodbc
import pandas as pd
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-VHHBRU7\\SQLEXPRESS;'
    'DATABASE=Northwind;'
    'Trusted_Connection=yes;'
)

# read to DataFrame
df = pd.read_sql("""
                 SELECT * FROM dbo.Employees where EmployeeID >3
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print('EmployeeID >3')
print(df)
print('\n')
print ('-------------------')
print('BirthDate > 1948-12-08')
df = pd.read_sql("""
                 SELECT * FROM dbo.Employees where BirthDate > '1948-12-08'
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

print(df)
print ('-------------------')
print(' Country != USA')
df = pd.read_sql("""
                 SELECT * FROM dbo.Employees where Country <> 'USA'
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

print(df)


conn.close()

EmployeeID >3
   EmployeeID   LastName FirstName                     Title TitleOfCourtesy  BirthDate   HireDate                           Address     City Region PostalCode Country       HomePhone Extension                                              Photo                                              Notes  ReportsTo                              PhotoPath
0           4    Peacock  Margaret      Sales Representative            Mrs. 1937-09-19 1993-05-03              4110 Old Redmond Rd.  Redmond     WA      98052     USA  (206) 555-8122      5176  b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...  Margaret holds a BA in English literature from...          2   http://accweb/emmployees/peacock.bmp
1           5   Buchanan    Steven             Sales Manager             Mr. 1955-03-04 1993-10-17                   14 Garrett Hill   London   None    SW1 8JR      UK   (71) 555-4848      3453  b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...  Steven Buchanan graduated from St. Andrews Uni..

C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\2630615203.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""
C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\2630615203.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""
C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\2630615203.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""


In [38]:

import pyodbc
import pandas as pd
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-VHHBRU7\\SQLEXPRESS;'
    'DATABASE=Northwind;'
    'Trusted_Connection=yes;'
)

# read to DataFrame
df = pd.read_sql("""
                 select * from Products where UnitPrice between 20 and 30
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print('UnitPrice between 20 and 30')
print(df)
print('\n')



print ('-------------------')
print('UnitPrice >= 20 AND UnitPrice <= 30')
df = pd.read_sql("""
                 select * from Products where  UnitPrice >= 20 AND UnitPrice <= 30
""", conn)

# all columns, no line wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

print(df)




conn.close()


UnitPrice between 20 and 30
    ProductID                       ProductName  SupplierID  CategoryID      QuantityPerUnit  UnitPrice  UnitsInStock  UnitsOnOrder  ReorderLevel  Discontinued
0           4      Chef Anton's Cajun Seasoning           2           2       48 - 6 oz jars      22.00            53             0             0         False
1           5            Chef Anton's Gumbo Mix           2           2             36 boxes      21.35             0             0             0          True
2           6      Grandma's Boysenberry Spread           3           2       12 - 8 oz jars      25.00           120             0            25         False
3           7   Uncle Bob's Organic Dried Pears           3           7      12 - 1 lb pkgs.      30.00            15             0            10         False
4          11                    Queso Cabrales           5           4            1 kg pkg.      21.00            22            30            30         False
5          1

C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\2927208819.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""
C:\Users\ysy\AppData\Local\Temp\ipykernel_9764\2927208819.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""
